In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from xgboost import XGBClassifier, plot_importance
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np


In [2]:
df1 = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
df1["proprosion"] = np.where(df1["cant_viewed_product"]>0, (df1["cant_checkouts"]/df1["cant_viewed_product"]),0)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 14 columns):
cant_conversions               38829 non-null float64
cant_checkouts                 38829 non-null float64
cant_viewed_product            38829 non-null float64
cant_searched_product          38829 non-null float64
cant_ad_campaign_hit           38829 non-null float64
ad_campaign_hit                38829 non-null bool
cant_lead                      38829 non-null float64
lead                           38829 non-null bool
cant_brand_listing             38829 non-null float64
cant_visited_site              38829 non-null float64
cant_generic_listing           38829 non-null float64
total_sesiones                 38829 non-null int64
promedio_eventos_por_sesion    38829 non-null float64
proprosion                     38829 non-null float64
dtypes: bool(2), float64(11), int64(1)
memory usage: 3.9+ MB


In [5]:
df2 = pd.read_csv('data/features_time.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('data/sku.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('data/features_compras.csv', low_memory=False, index_col='person')


######################################################################################

df5 = pd.read_csv('data/vistas_promedio.csv', low_memory=False, index_col='person')

df7 = pd.read_csv('data/features_mas_vistos.csv', low_memory=False, index_col='person')

df8 = pd.read_csv('data/features_checkout.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('data/paises_features.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('data/region_features.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('data/static.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('data/events_fe.csv', low_memory=False, index_col='person')
df13 = pd.read_csv('data/channel_device.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('data/check.csv', low_memory=False, index_col='person')
df15 = pd.read_csv('data/view_sku.csv', low_memory=False, index_col='person')
df16 = pd.read_csv('data/event_por_hora.csv', low_memory=False, index_col='person')
df17 = pd.read_csv('data/ult_hora.csv', low_memory=False, index_col='person')


df_unidos = df1.join(df2).fillna(0)
df_unidos = df_unidos.join(df3).fillna(0)
df_unidos = df_unidos.join(df4).fillna(0)
df_unidos = df_unidos.join(df5).fillna(0)

df_unidos = df_unidos.join(df7).fillna(0)
df_unidos = df_unidos.join(df8).fillna(0)
df_unidos = df_unidos.join(df9).fillna(0)
df_unidos = df_unidos.join(df10).fillna(0)
df_unidos = df_unidos.join(df11).fillna(0)
df_unidos = df_unidos.join(df12).fillna(0)
df_unidos = df_unidos.join(df13).fillna(0)
df_unidos = df_unidos.join(df14).fillna(0)
df_unidos = df_unidos.join(df15).fillna(0)
df_unidos = df_unidos.join(df16).fillna(0)
df_unidos = df_unidos.join(df17).fillna(0)

df_unidos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Columns: 101 entries, cant_conversions to hour
dtypes: bool(2), float64(34), int64(65)
memory usage: 30.9+ MB


In [6]:
labels = pd.read_csv('data/labels_training_set.csv', low_memory=False, index_col='person')


df_test = df_unidos.join(labels, how= "inner")
df_test.head()

,cant_conversions,cant_checkouts,cant_viewed_product,cant_searched_product,cant_ad_campaign_hit,ad_campaign_hit,cant_lead,lead,cant_brand_listing,cant_visited_site,...,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour,label
person,,,,,,,,,,,,,,,,,,,,,
ad93850f,0.0,1.0,20.0,0.0,10.0,True,0.0,False,15.0,5.0,...,0,0,0,0,0,0,1,1,0,0
1b9f7cf6,0.0,1.0,9.0,8.0,0.0,False,0.0,False,3.0,3.0,...,0,0,0,1,0,0,0,0,19,0
de8fe91b,0.0,1.0,27.0,13.0,0.0,False,0.0,False,3.0,2.0,...,0,0,0,0,0,0,1,0,22,0
45baf068,0.0,1.0,6.0,0.0,5.0,True,0.0,False,3.0,1.0,...,0,0,0,0,0,0,0,0,0,0
99abca5a,2.0,18.0,630.0,5.0,12.0,True,1.0,True,30.0,56.0,...,1,1,1,0,1,1,1,1,1,0


In [7]:
X, y = df_test.iloc[:,:-1],df_test.iloc[:,-1]  #X van los features, en y el label, asumo que esta bien esto

In [8]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [10]:
#y_train

In [11]:
param_grid = { 
    'n_estimators': [100,200,300,400,500],
    'learning_rate' : [0.1,0.01],
    'max_depth' : [1,2,3,4,5],
    'scale_pos_weight': [1,2,3,4,5,6]
} 

In [12]:
from sklearn.model_selection import GridSearchCV 
xgb = XGBClassifier(random_state=80)
CV_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid, cv= 5, scoring='roc_auc', n_jobs=-1, verbose=10) 
CV_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
[CV] n_estimators=100, scale_pos_weight=1, learning_rate=0.1, max_depth=1 
[CV] n_estimators=100, scale_pos_weight=1, learning_rate=0.1, max_depth=1 
[CV] n_estimators=100, scale_pos_weight=1, learning_rate=0.1, max_depth=1 
[CV] n_estimators=100, scale_pos_weight=1, learning_rate=0.1, max_depth=1 
[CV]  n_estimators=100, scale_pos_weight=1, learning_rate=0.1, max_depth=1, score=0.852682590357, total=   6.4s
[CV]  n_estimators=100, scale_pos_weight=1, learning_rate=0.1, max_depth=1, score=0.884387920112, total=   6.4s
[CV] n_estimators=100, scale_pos_weight=1, learning_rate=0.1, max_depth=1 
[CV]  n_estimators=100, scale_pos_weight=1, learning_rate=0.1, max_depth=1, score=0.838698044023, total=   6.3s
[CV] n_estimators=100, scale_pos_weight=2, learning_rate=0.1, max_depth=1 
[CV]  n_estimators=100, scale_pos_weight=1, learning_rate=0.1, max_depth=1, score=0.860961541268, total=   6.4s
[CV] n_estimators=100, scale_pos_weigh

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   13.4s


[CV]  n_estimators=100, scale_pos_weight=2, learning_rate=0.1, max_depth=1, score=0.861356572422, total=   6.3s
[CV] n_estimators=100, scale_pos_weight=3, learning_rate=0.1, max_depth=1 
[CV] n_estimators=100, scale_pos_weight=3, learning_rate=0.1, max_depth=1 
[CV]  n_estimators=100, scale_pos_weight=2, learning_rate=0.1, max_depth=1, score=0.839967563669, total=   6.6s
[CV]  n_estimators=100, scale_pos_weight=2, learning_rate=0.1, max_depth=1, score=0.887173162516, total=   6.5s
[CV] n_estimators=100, scale_pos_weight=3, learning_rate=0.1, max_depth=1 
[CV]  n_estimators=100, scale_pos_weight=3, learning_rate=0.1, max_depth=1, score=0.853149381502, total=   6.5s
[CV] n_estimators=100, scale_pos_weight=3, learning_rate=0.1, max_depth=1 


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   20.3s


[CV] n_estimators=100, scale_pos_weight=3, learning_rate=0.1, max_depth=1 
[CV]  n_estimators=100, scale_pos_weight=3, learning_rate=0.1, max_depth=1, score=0.883307423646, total=   6.5s
[CV] n_estimators=100, scale_pos_weight=4, learning_rate=0.1, max_depth=1 
[CV]  n_estimators=100, scale_pos_weight=3, learning_rate=0.1, max_depth=1, score=0.861756815069, total=   6.4s
[CV] n_estimators=100, scale_pos_weight=4, learning_rate=0.1, max_depth=1 
[CV]  n_estimators=100, scale_pos_weight=3, learning_rate=0.1, max_depth=1, score=0.840628380956, total=   6.4s
[CV]  n_estimators=100, scale_pos_weight=3, learning_rate=0.1, max_depth=1, score=0.887086117718, total=   6.2s
[CV] n_estimators=100, scale_pos_weight=4, learning_rate=0.1, max_depth=1 
[CV]  n_estimators=100, scale_pos_weight=4, learning_rate=0.1, max_depth=1, score=0.853021222326, total=   6.2s
[CV] n_estimators=100, scale_pos_weight=4, learning_rate=0.1, max_depth=1 
[CV] n_estimators=100, scale_pos_weight=4, learning_rate=0.1, max

...

[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 186.1min


[CV]  n_estimators=400, scale_pos_weight=3, learning_rate=0.01, max_depth=5, score=0.855891779475, total=  58.4s
[CV] n_estimators=400, scale_pos_weight=3, learning_rate=0.01, max_depth=5 
[CV]  n_estimators=400, scale_pos_weight=3, learning_rate=0.01, max_depth=5, score=0.876149264804, total=  58.3s
[CV] n_estimators=400, scale_pos_weight=4, learning_rate=0.01, max_depth=5 
[CV]  n_estimators=400, scale_pos_weight=3, learning_rate=0.01, max_depth=5, score=0.857728331134, total=  58.5s
[CV] n_estimators=400, scale_pos_weight=4, learning_rate=0.01, max_depth=5 
[CV]  n_estimators=400, scale_pos_weight=3, learning_rate=0.01, max_depth=5, score=0.827331778224, total=  58.6s
[CV] n_estimators=400, scale_pos_weight=4, learning_rate=0.01, max_depth=5 
[CV]  n_estimators=400, scale_pos_weight=3, learning_rate=0.01, max_depth=5, score=0.883096739281, total=  58.3s
[CV] n_estimators=400, scale_pos_weight=4, learning_rate=0.01, max_depth=5 
[CV]  n_estimators=400, scale_pos_weight=4, learning_ra

[CV] n_estimators=500, scale_pos_weight=6, learning_rate=0.01, max_depth=5 
[CV]  n_estimators=500, scale_pos_weight=5, learning_rate=0.01, max_depth=5, score=0.877761837044, total= 1.3min
[CV] n_estimators=500, scale_pos_weight=6, learning_rate=0.01, max_depth=5 
[CV]  n_estimators=500, scale_pos_weight=6, learning_rate=0.01, max_depth=5, score=0.850778957722, total= 1.3min
[CV] n_estimators=500, scale_pos_weight=6, learning_rate=0.01, max_depth=5 
[CV]  n_estimators=500, scale_pos_weight=6, learning_rate=0.01, max_depth=5, score=0.869765270739, total= 1.3min
[CV]  n_estimators=500, scale_pos_weight=6, learning_rate=0.01, max_depth=5, score=0.863640248567, total= 1.3min
[CV]  n_estimators=500, scale_pos_weight=6, learning_rate=0.01, max_depth=5, score=0.827331778224, total=  57.3s
[CV]  n_estimators=500, scale_pos_weight=6, learning_rate=0.01, max_depth=5, score=0.876790710747, total=  55.3s


[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 200.1min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=80, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [100, 200, 300, 400, 500], 'scale_pos_weight': [1, 2, 3, 4, 5, 6], 'learning_rate': [0.1, 0.01], 'max_depth': [1, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=10)

In [11]:
CV_xgb.best_params_ #aca te dice el mejor hiperparametro


{'learning_rate': 0.01,
 'max_depth': 3,
 'n_estimators': 400,
 'scale_pos_weight': 6}

In [20]:
xg_reg = XGBClassifier(max_depth=4,learning_rate=0.01,n_estimators=200,scale_pos_weight = 3,silent=True,n_jobs=-1)
#falta jugar con los hiperparametros
# Voting con multinomialNB, RF, y SVC (no lineal), probar las distintas combinaciones

In [21]:
xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=3,
       seed=None, silent=True, subsample=1)

In [22]:
score=xg_reg.score(X_test,y_test)
#print (X_test,y_test)
print(score)

Y_pred = xg_reg.predict(X_test)
print(roc_auc_score(y_test, Y_pred))
#0.953901622457
#0.550046837876

0.94076744785
0.577716839918


In [23]:
feature_importances = pd.DataFrame(xg_reg.feature_importances_,\
                                   index = X_train.columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances.head(50)

,importance
ult_check,0.110220
ult_conv,0.103206
sum_timestamp,0.099198
ult_view,0.087174
cant_viewed_product,0.051770
timestamp,0.045758
cant_conversions,0.040414
cant_event,0.039412
cant_brand_listing,0.038744
cant_checkouts,0.038410


In [24]:
df_submit = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos)


kaggle_pred = xg_reg.predict_proba(df_events)
proba_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_comprar)
df_submit['label'] = series.values

df_submit.to_csv('submit/submit_xg_clase.csv')

In [25]:
len(df_events)

19415

In [26]:
df_submit["label"].mean()

0.1698857891641724